In [1]:
import pandas as pd
import time

import requests
from bs4 import BeautifulSoup
import praw
from praw.models import MoreComments

from os.path import exists

In [239]:
## for downloading all website of daily general discussion
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument("--disable-notifications")

bot = webdriver.Chrome(ChromeDriverManager().install(), options=options)
#bot = webdriver.Chrome()

url = 'https://www.reddit.com/r/CryptoCurrency/search/?q=title%3A%22Daily%20Discussion%20-%20March%22%20&restrict_sr=1&sr_nsfw='
bot.get(url)
#waiting for the page to load
time.sleep(3) 
#repeat scrolling
for i in range(25):
    #scroll 300 px
    bot.execute_script('window.scrollTo(0,(document.body.scrollHeight))')
    #waiting for the page to load
    time.sleep(3) 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\ricky\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [240]:
# req = requests.get(reqUrl, 'html.parser')

In [241]:
soup = BeautifulSoup(bot.page_source, 'html.parser')

In [242]:
urls = []
for link in soup.find_all('a'):
    #print(link.get('href'))
    if 'comments' in link.get('href'):
        urls.append('https://www.reddit.com' + link.get('href'))
    #urls.append(link.get('href'))
#urls = [u for u in urls if 'Daily' in urls]

In [243]:
len(urls)

121

In [244]:
#urls

In [245]:
with open(r'url.txt', 'w') as fp:
    for item in urls:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [246]:
bot.close()
bot.quit()

In [251]:
# list of url
dfs = pd.read_excel('url.xlsx', sheet_name='Sheet1')
#dfs[dfs.year==2022].url.tolist()

In [2]:
# reddit agent
reddit = praw.Reddit(
    client_id="x8WY3ixCludkwhClM6wTTQ",
    client_secret="niFo_CJBwLKgS_bmUssOgWTKEPMCQw",
    user_agent="DASC7600",)
print(reddit.read_only)

True


In [279]:
targetUrl = dfs[dfs.year==2022].url.tolist()[1]
targetUrl

'https://www.reddit.com/r/CryptoCurrency/comments/tuwz77/daily_discussion_april_3_2022_gmt0/'

In [21]:
targetUrl='https://old.reddit.com/r/CryptoCurrency/comments/w1l73o/daily_general_discussion_july_18_2022_gmt0/'
sub = []

submission = reddit.submission(url=str(targetUrl))
submission.comments.replace_more(limit=100)
for comment in submission.comments.list():
        sub.append([comment.score,comment.body,comment.created])

In [22]:
posts = pd.DataFrame(sub,columns=['score','body','created'])

In [23]:
import datetime as dt

seconds = posts["created"] # Generating test data

start = dt.datetime.utcfromtimestamp(0) # You need a reference point GMT+0

datetime_series = seconds.astype('timedelta64[s]') + start

posts["created_date"] = datetime_series.dt.date
posts["created_time"] = datetime_series.dt.time

posts.sort_values('created_time')

#2386+1882

,score,body,created,created_date,created_time
555,1,[removed],1.658102e+09,2022-07-18,00:01:38
115,4,Let's have a green week to come!,1.658103e+09,2022-07-18,00:02:06
277,2,Good news would be greatly appreciated this week.,1.658103e+09,2022-07-18,00:02:38
1247,1,Good sign... I'll buy more ETH,1.658103e+09,2022-07-18,00:03:12
464,2,"Like clockwork, a Sunday night dump, back to $...",1.658103e+09,2022-07-18,00:04:48
...,...,...,...,...,...
155,1,Guys you seem a bit too excited today about a ...,1.658163e+09,2022-07-18,16:50:53
354,1,Have 50% in for this run up. 50% on the side j...,1.658163e+09,2022-07-18,16:51:26
611,1,"Just got messaged by a ""Crypto Analyst."" It ga...",1.658163e+09,2022-07-18,16:51:33
612,1,"Up or not up, there is no down.",1.658163e+09,2022-07-18,16:51:47


In [317]:
#sub = []
for i,targetUrl,m,d,y,*_ in list(dfs[dfs.year>=2020].itertuples(index=True, name=None)):
    if not exists('redditData/' + str(int(y)) + "_" + m + str(int(d)) + '.csv'):
        sub = []
        submission = reddit.submission(url=str(targetUrl))
        submission.comments.replace_more(limit=100)
        for comment in submission.comments.list():
            sub.append([comment.score,comment.body,comment.created])
        posts = pd.DataFrame(sub,columns=['score','body','created'])
        posts.to_csv('redditData/' + str(int(y)) + "_" + m + str(int(d)) + '.csv')
#posts = pd.DataFrame(sub,columns=['score','body','created'])
#posts.to_csv('2022.csv')

True

In [315]:
y


2022.0